In [ ]:
import torch
from torch import nn

In [ ]:
data = torch.tensor([[20,100000,2], [25,80000,1], [30,20000,3], [31, 200000,3], [39, 350000,2], [40, 400000,2]], dtype=torch.float)

In [ ]:
data

tensor([[2.0000e+01, 1.0000e+05, 2.0000e+00],
        [2.5000e+01, 8.0000e+04, 1.0000e+00],
        [3.0000e+01, 2.0000e+04, 3.0000e+00],
        [3.1000e+01, 2.0000e+05, 3.0000e+00],
        [3.9000e+01, 3.5000e+05, 2.0000e+00],
        [4.0000e+01, 4.0000e+05, 2.0000e+00]])

In [ ]:
X = data[:, :-1]
y = data[:, -1].unsqueeze(dim=1)

In [ ]:
X,y

(tensor([[2.0000e+01, 1.0000e+05],
         [2.5000e+01, 8.0000e+04],
         [3.0000e+01, 2.0000e+04],
         [3.1000e+01, 2.0000e+05],
         [3.9000e+01, 3.5000e+05],
         [4.0000e+01, 4.0000e+05]]),
 tensor([[2.],
         [1.],
         [3.],
         [3.],
         [2.],
         [2.]]))

In [ ]:
# Min-Max Scaling
X_min = X.min(axis=0)
X_max = X.max(axis=0)
X_minmax_norm = (X - X_min.values) / (X_max.values - X_min.values)

# Z-Score Normalization
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_zscore_norm = (X - X_mean) / X_std

In [ ]:
def train_loop(model:nn.Module,
               X:torch.tensor,
               y:torch.tensor,
               loss_fn:nn.Module,
               optimizer:torch.optim,
               device:torch.device):

  X = X.to(device)
  y = y.to(device)
  model = model.to(device)

  #1.forward pass
  model.train()
  y_hat = model(X)
  # print(y)

  #2.loss calculation
  loss = loss_fn(y_hat,y)

  #3.optimizer zero_grad()
  optimizer.zero_grad()

  #4.loss.backward()
  loss.backward()

  #5.optimizer step
  optimizer.step()

  return loss

In [ ]:
class LinearRegression(nn.Module):
  def __init__(self):
    super().__init__()

    self.layer = nn.Linear(in_features=2, out_features=1)

  def forward(self,x):
    return self.layer(x)

In [ ]:
torch.manual_seed(42)
model = LinearRegression()

In [ ]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(
    model.parameters(),
    lr = 0.1
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
epochs = 100

for epoch in range(epochs):
  train_loss = train_loop(
      model=model,
      X=X_minmax_norm,
      y=y,
      loss_fn = loss_fn,
      optimizer=optimizer,
      device = device
  )
  if epoch % 10 == 0:
    print(f"Epoch:{epoch} | Loss:{train_loss}")


Epoch:0 | Loss:3.7542121410369873
Epoch:10 | Loss:1.2975142002105713
Epoch:20 | Loss:0.9496739506721497
Epoch:30 | Loss:0.6388378739356995
Epoch:40 | Loss:0.4722718298435211
Epoch:50 | Loss:0.4225962162017822
Epoch:60 | Loss:0.4018150269985199
Epoch:70 | Loss:0.39471086859703064
Epoch:80 | Loss:0.3904346227645874
Epoch:90 | Loss:0.38618502020835876
